<a href="https://colab.research.google.com/github/casllmproject/wjs_llm_project/blob/main/german_gpt_4o_survey_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount goolge drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install openai
!pip install openai==0.28
!pip install openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.108.0
    Uninstalling openai-1.108.0:
      Successfully uninstalled openai-1.108.0


In [ ]:
import pandas as pd
import openai
import numpy as np
import json
import time
import os
import re
import random
from tqdm import tqdm

In [ ]:
openai.api_key = open("/content/drive/MyDrive/CASLLM/casllm_apikey.txt", "r").read().strip("\n")

In [ ]:
# Load the survey questions from the CSV file
survey_df = pd.read_csv('/content/drive/MyDrive/AI_shoes/german_survey_questions.csv')

In [ ]:
def gpt_take_survey(model_name, temperature=0.7, max_retries=3):
    responses = []

    # Extract the context from the CSV file
    context = survey_df[survey_df['Variable_Name'] == 'context']['Question'].values[0]

    # Track time
    start_time = time.time()

    # Loop through each survey question with a progress bar
    for i, row in tqdm(enumerate(survey_df.iterrows()), total=len(survey_df), desc=f'Processing {model_name}'):
        if row[1]['Variable_Name'] == 'context':
            continue  # Skip the context row

        # Prepare the prompt
        prompt = f"{context}\n\n{row[1]['Question']}\nOptions: {row[1]['Options']}\nChoose one of the options."

        # Implement retry mechanism
        for attempt in range(max_retries):
            try:
                # Make the API call using ChatCompletion
                response = openai.ChatCompletion.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": context},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=temperature
                )

                # Extract the model's response
                answer = response['choices'][0]['message']['content'].strip()

                # Extract the number from the response using regular expression
                match = re.search(r'\b[1-5]\b', answer)
                if match:
                    answer = match.group(0)
                else:
                    answer = None  # Handle cases where the model's response doesn't match any option

                # Store the response
                responses.append({
                    "Model": model_name,
                    "Variable_Name": row[1]['Variable_Name'],
                    "Question": row[1]['Question'],
                    "Response": answer
                })
                break  # If successful, break out of retry loop

            except openai.error.APIError as e:
                print(f"Attempt {attempt + 1} failed with error: {e}")
                time.sleep(random.uniform(1, 3))  # Wait a bit before retrying

        # Update progress and time estimate
        elapsed_time = time.time() - start_time
        questions_remaining = len(survey_df) - i - 1
        time_per_question = elapsed_time / (i + 1)
        time_remaining = questions_remaining * time_per_question
        tqdm.write(f"Estimated time remaining: {time.strftime('%H:%M:%S', time.gmtime(time_remaining))}")

    # Convert the responses to a DataFrame and save to CSV in the specified folder
    gpt_df = pd.DataFrame(responses)
    output_path = f'/content/drive/MyDrive/AI_shoes/german_{model_name}_responses.csv'
    gpt_df.to_csv(output_path, index=False)

    print(f"Responses from {model_name} saved to {output_path}.")

# Example: Run the survey with GPT-4o-mini, GPT-4o
models = ['gpt-4o-2024-08-06']
for model in models:
    gpt_take_survey(model)

Processing gpt-4o-2024-08-06:   5%|▌         | 2/40 [00:02<00:55,  1.47s/it]

Estimated time remaining: 00:00:55


Processing gpt-4o-2024-08-06:   8%|▊         | 3/40 [00:07<01:45,  2.86s/it]

Estimated time remaining: 00:01:35


Processing gpt-4o-2024-08-06:  10%|█         | 4/40 [00:10<01:45,  2.93s/it]

...

Estimated time remaining: 00:00:06


Processing gpt-4o-2024-08-06:  98%|█████████▊| 39/40 [01:58<00:02,  2.73s/it]

Estimated time remaining: 00:00:03


Processing gpt-4o-2024-08-06: 100%|██████████| 40/40 [02:01<00:00,  3.03s/it]

Estimated time remaining: 00:00:00
Responses from gpt-4o-2024-08-06 saved to /content/drive/MyDrive/AI_shoes/german_gpt-4o-2024-08-06_responses.csv.


In [ ]:
gpt4o_flag = pd.read_csv("/content/drive/MyDrive/AI_shoes/german_gpt-4o-2024-08-06_responses.csv", encoding='utf-8')
print(gpt4o_flag)

In [ ]:
def gpt_take_survey(model_name, temperature=0.7, max_retries=3):
    responses = []

    # Extract the context from the CSV file
    context = survey_df[survey_df['Variable_Name'] == 'context']['Question'].values[0]

    # Track time
    start_time = time.time()

    # Loop through each survey question with a progress bar
    for i, row in tqdm(enumerate(survey_df.iterrows()), total=len(survey_df), desc=f'Processing {model_name}'):
        if row[1]['Variable_Name'] == 'context':
            continue  # Skip the context row

        # Prepare the prompt
        prompt = f"{context}\n\n{row[1]['Question']}\nOptions: {row[1]['Options']}\nChoose one of the options."

        # Implement retry mechanism
        for attempt in range(max_retries):
            try:
                # Make the API call using ChatCompletion
                response = openai.ChatCompletion.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": context},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=temperature
                )

                # Extract the model's response
                answer = response['choices'][0]['message']['content'].strip()

                # Extract the number from the response using regular expression
                match = re.search(r'\b[1-5]\b', answer)
                if match:
                    answer = match.group(0)
                else:
                    answer = None  # Handle cases where the model's response doesn't match any option

                # Store the response
                responses.append({
                    "Model": model_name,
                    "Variable_Name": row[1]['Variable_Name'],
                    "Question": row[1]['Question'],
                    "Response": answer
                })
                break  # If successful, break out of retry loop

            except openai.error.APIError as e:
                print(f"Attempt {attempt + 1} failed with error: {e}")
                time.sleep(random.uniform(1, 3))  # Wait a bit before retrying

        # Update progress and time estimate
        elapsed_time = time.time() - start_time
        questions_remaining = len(survey_df) - i - 1
        time_per_question = elapsed_time / (i + 1)
        time_remaining = questions_remaining * time_per_question
        tqdm.write(f"Estimated time remaining: {time.strftime('%H:%M:%S', time.gmtime(time_remaining))}")

    # Convert the responses to a DataFrame and save to CSV in the specified folder
    gpt_df = pd.DataFrame(responses)
    output_path = f'/content/drive/MyDrive/AI_shoes/german_{model_name}_responses_2nd.csv'
    gpt_df.to_csv(output_path, index=False)

    print(f"Responses from {model_name} saved to {output_path}.")

# Example: Run the survey with GPT-4o-mini, GPT-4o
models = ['gpt-4o-2024-08-06']
for model in models:
    gpt_take_survey(model)

Processing gpt-4o-2024-08-06:   5%|▌         | 2/40 [00:02<00:53,  1.41s/it]

Estimated time remaining: 00:00:53


Processing gpt-4o-2024-08-06:   8%|▊         | 3/40 [00:04<00:55,  1.50s/it]

Estimated time remaining: 00:00:54


Processing gpt-4o-2024-08-06:  10%|█         | 4/40 [00:07<01:20,  2.25s/it]

...

Estimated time remaining: 00:00:05


Processing gpt-4o-2024-08-06:  98%|█████████▊| 39/40 [01:49<00:03,  3.37s/it]

Estimated time remaining: 00:00:02


Processing gpt-4o-2024-08-06: 100%|██████████| 40/40 [01:50<00:00,  2.77s/it]

Estimated time remaining: 00:00:00
Responses from gpt-4o-2024-08-06 saved to /content/drive/MyDrive/AI_shoes/german_gpt-4o-2024-08-06_responses_2nd.csv.


In [ ]:
gpt4o_flag = pd.read_csv("/content/drive/MyDrive/AI_shoes/german_gpt-4o-2024-08-06_responses_2nd.csv", encoding='utf-8')
print(gpt4o_flag)